In [1]:
import tensorflow as tf
from tensorflow.keras.applications import densenet
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.layers import Dense, Dropout, Input, Conv2D
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
import cv2
import tensorflow as tf
import re
import pickle
from PIL import Image
from skimage.transform import resize
import warnings

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [3]:
def get_random_set():
    obj = reqd[np.random.randint(len(reqd))]
    for i in range(len(obj["imgs"])):
        plt.subplot(1, len(obj["imgs"]),i+1)
        plt.axis('off')
        img = plt.imread(get_img_path(obj["imgs"][i]))
        plt.imshow(img)
    print(obj["desc"])

def get_random_set2(df, k):
    obj = df[k]
    for i in range(len(obj["imgs"])):
        plt.subplot(1, len(obj["imgs"]),i+1)
        plt.axis('off')
        img = plt.imread(get_img_path(obj["imgs"][i]))
        plt.imshow(img)
    print(obj["desc"])
    
def get_img_path(x):
    return "./imgs/"+x+".png"

In [4]:
train_dataset = pd.read_csv('df_train.csv')
test_dataset = pd.read_csv('df_test.csv')
cv_dataset = pd.read_csv('df_cv.csv')
train_dataset = train_dataset.iloc[:,:-3]
test_dataset = test_dataset.iloc[:,:-3]
cv_dataset = cv_dataset.iloc[:,:-3]

In [5]:
train_dataset.shape

(2068, 4)

In [6]:
train_dataset

,Unnamed: 0,desc,img1,img2
0,0,startseq right upper lobe airspace disease con...,CXR3069_IM-1432-1001,CXR3069_IM-1432-2001
1,1,startseq mediastinal contours are normal lungs...,CXR2629_IM-1116-1001,CXR2629_IM-1116-2001
2,2,startseq cardiomediastinal contours are unchan...,CXR1888_IM-0576-1001,CXR1888_IM-0576-4004
3,3,startseq heart size normal the lungs are clear...,CXR2248_IM-0844-1001,CXR2248_IM-0844-1002
4,4,startseq the cardiomediastinal contours are wi...,CXR3408_IM-1648-1001,CXR3408_IM-1648-1002
...,...,...,...,...
2063,2331,startseq the heart normal size the mediastinum...,CXR2243_IM-0840-1001,CXR2243_IM-0840-2001
2064,2332,startseq normal heart size normal mediastinal ...,CXR1356_IM-0231-1001,CXR1356_IM-0231-2001
2065,2333,startseq frontal and lateral views the chest s...,CXR1883_IM-0572-1001,CXR1883_IM-0572-2001
2066,2334,startseq normal heart size and mediastinal con...,CXR2622_IM-1110-1001,CXR2622_IM-1110-1002


In [7]:
def load_image(img_name):
    image = Image.open(img_name)
    X = np.asarray(image.convert("RGB"))
    X = np.asarray(X)
    X = preprocess_input(X)
    X = resize(X, (224,224,3))
    X = np.expand_dims(X, axis=0)
    X = np.asarray(X)
    
    return X

In [8]:
chex = densenet.DenseNet121(include_top=False, weights = None, input_shape=(224,224,3), pooling="avg")

In [9]:
X = chex.output
X = Dense(14, activation="sigmoid", name="predictions")(X)
model = Model(inputs=chex.input, outputs=X)

In [10]:
model.load_weights(r'./utils/Image_features_enc.h5')

In [11]:
model = Model(inputs = model.input, outputs = model.layers[-2].output)

In [12]:
img1 = model.predict(load_image(get_img_path(train_dataset.iloc[0]["img1"])))
img2 = model.predict(load_image(get_img_path(train_dataset.iloc[0]["img2"])))

In [13]:
img1

array([[6.8697002e-04, 1.3616269e-03, 3.8314806e-03, ..., 8.3689243e-01,
        8.2594419e-01, 6.8347275e-01]], dtype=float32)

In [14]:
input_ = np.concatenate((img1, img2), axis=1)
# input_ = tf.reshape(input_, (input_.shape[0], -1, input_.shape[-1]))
input_.shape


(1, 2048)

In [15]:
def image_features(train, test, cv):
    Xnet_features_attention = {}
    
    for i, item in enumerate(tqdm(train.values)):
        key = "train_" + str(i)
        img1 = load_image(get_img_path(item[2]))
        img1_features = model.predict(img1)
        
        img2 = load_image(get_img_path(item[3]))
        img2_features = model.predict(img2)

        input_ = np.concatenate((img1_features, img2_features), axis=1)
#         input_ = tf.reshape(input_, (input_.shape[0], -1, input_.shape[-1]))

        Xnet_features_attention[key] = input_
    
    for i, item in enumerate(tqdm(test.values)):
        key = "test_" + str(i)
        img1 = load_image(get_img_path(item[2]))
        img1_features = model.predict(img1)
        
        img2 = load_image(get_img_path(item[3]))
        img2_features = model.predict(img2)

        input_ = np.concatenate((img1_features, img2_features), axis=1)
#         input_ = tf.reshape(input_, (input_.shape[0], -1, input_.shape[-1]))

        Xnet_features_attention[key] = input_

    for i, item in enumerate(tqdm(cv.values)):
        key = "cv_" + str(i)
        img1 = load_image(get_img_path(item[2]))
        img1_features = model.predict(img1)
        
        img2 = load_image(get_img_path(item[3]))
        img2_features = model.predict(img2)

        input_ = np.concatenate((img1_features, img2_features), axis=1)
#         input_ = tf.reshape(input_, (input_.shape[0], -1, input_.shape[-1]))

        Xnet_features_attention[key] = input_
        
    return Xnet_features_attention

In [16]:
Xnet_features_attention = image_features(train_dataset, test_dataset, cv_dataset)

100%|██████████| 444/444 [02:07<00:00,  3.49it/s]


In [18]:
Xnet_features_attention.keys()

dict_keys(['train_0', 'train_1', 'train_2', 'train_3', 'train_4', 'train_5', 'train_6', 'train_7', 'train_8', 'train_9', 'train_10', 'train_11', 'train_12', 'train_13', 'train_14', 'train_15', 'train_16', 'train_17', 'train_18', 'train_19', 'train_20', 'train_21', 'train_22', 'train_23', 'train_24', 'train_25', 'train_26', 'train_27', 'train_28', 'train_29', 'train_30', 'train_31', 'train_32', 'train_33', 'train_34', 'train_35', 'train_36', 'train_37', 'train_38', 'train_39', 'train_40', 'train_41', 'train_42', 'train_43', 'train_44', 'train_45', 'train_46', 'train_47', 'train_48', 'train_49', 'train_50', 'train_51', 'train_52', 'train_53', 'train_54', 'train_55', 'train_56', 'train_57', 'train_58', 'train_59', 'train_60', 'train_61', 'train_62', 'train_63', 'train_64', 'train_65', 'train_66', 'train_67', 'train_68', 'train_69', 'train_70', 'train_71', 'train_72', 'train_73', 'train_74', 'train_75', 'train_76', 'train_77', 'train_78', 'train_79', 'train_80', 'train_81', 'train_82', 'tr

In [19]:
# save the file for future use
f = open('Image_features_attention.pickle','wb')
pickle.dump(Xnet_features_attention, f)
f.close()

In [20]:
len(train_dataset)

2068

In [ ]:
for item in tqdm(train_dataset.values):
    print(item)
    break

In [ ]:
train_dataset.iloc[10:30]